### Установка библиотеки beautifulsoup4

In [1]:
!pip install beautifulsoup4

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Импорт библиотек для осуществления запросов и парсинга, а также для работы с таблицами

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

В url поставить ссылку из поисковика

In [3]:
url = 'https://www.google.ru/search?newwindow=1&hl=en&sxsrf=ALeKk003DHgB6VZvvzgoN_JOAIlqU5186Q:1592421448626&q=%D1%88%D0%BA%D0%BE%D0%BB%D0%B0+%D0%B2+%D0%B2%D0%BE%D0%BB%D0%BE%D0%B3%D0%B4%D0%B5&npsic=0&rflfq=1&rlha=0&rllag=59205422,39881362,1332&tbm=lcl&ved=2ahUKEwiDnJyWyInqAhVlxIsKHSvLBQwQjGp6BAgLEDI&rldoc=1#rlfi=hd:;si:;mv:[[59.2300572,39.945501],[59.182565800000006,39.811011099999995]];tbs:lrf:!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e16!3sIAE,lf:1,lf_ui:2'

Скачиваем страницу

In [4]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}

response = requests.get(url, headers=headers, allow_redirects=True)


page_text = ''
if response.status_code == 200:
    print('Запрос прошел успешно!')
    page_text = response.text
else:
    print(f'Ошибка! Response code: {response.status_code}')
    print(f'URL: {url}')
    print('--------------')

Ошибка! Response code: 429
URL: https://www.google.ru/search?newwindow=1&hl=en&sxsrf=ALeKk003DHgB6VZvvzgoN_JOAIlqU5186Q:1592421448626&q=%D1%88%D0%BA%D0%BE%D0%BB%D0%B0+%D0%B2+%D0%B2%D0%BE%D0%BB%D0%BE%D0%B3%D0%B4%D0%B5&npsic=0&rflfq=1&rlha=0&rllag=59205422,39881362,1332&tbm=lcl&ved=2ahUKEwiDnJyWyInqAhVlxIsKHSvLBQwQjGp6BAgLEDI&rldoc=1#rlfi=hd:;si:;mv:[[59.2300572,39.945501],[59.182565800000006,39.811011099999995]];tbs:lrf:!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e16!3sIAE,lf:1,lf_ui:2
--------------


In [5]:
#print(page_text[:1000])

"Размечаем" страницу

In [6]:
def BS(page_text):
    page_bs = None
    page_bs = BeautifulSoup(page_text)
    return page_bs


page_bs = BS(page_text)

Вот тут ищем ссылки на следующие страницы запроса

In [7]:
def other_links(page):
    link_containers = page.find_all('a', {'class': 'fl'})
    search_list = []
    for link in link_containers:
        search_list.append("https://www.google.ru"+link['href'])
    return search_list
        
search_list = other_links(page_bs)

print(search_list)

[]


Берем основную страницу, находим организации и их колличество

In [8]:
phones_list = []
def parse(page):
    shelf_containers = page.find_all('a', {'class': 'C8TUKc rllt__link a-no-hover-decoration'})
    print(f'Количество найденных блоков: {len(shelf_containers)}')
    #shelf_containers[0]
    for shelf in shelf_containers:
        items = shelf.find_all('span')
        for item in items:
            #17 - колличество знаков в номере
            if len(item.text)>16:
                   if item.text[0] == '8' and item.text[1] == ' ' and item.text[2] == '(':
                        phones_list.append(item.text)
    return (phones_list)

                
phones_list = parse(page_bs)
print(phones_list)

Количество найденных блоков: 0
[]


А теперь найдем информацию в остальных 

In [33]:
def last_num(link_list):
    num = str(link_list[-1]).split('start=')[1]
    num = int(num.split('&')[0])
    return num


def next_link(searches):
    for search in searches:
        url = str(search)
        #print(url)
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
        headers = {'User-Agent': user_agent}
        response = requests.get(url, headers=headers, allow_redirects=True)
        page_text = ''
        if response.status_code == 200:
            print('Запрос прошел успешно!')
            page_bs = BS(response.text)          
            parse(page_bs)
            search_list = other_links(page_bs)
            if search_list[-1] != searches[-1] and last_num(search_list) > last_num(searches):
                print (last_num(search_list)," >",last_num(searches))
                searches.append(search_list[-1])
        else:
            print(f'Ошибка! Response code: {response.status_code}')
            print(f'URL: {url}')
            print('--------------')
    
        
next_link(search_list)

Запрос прошел успешно!
Количество найденных блоков: 18
Запрос прошел успешно!
Количество найденных блоков: 20
Запрос прошел успешно!
Количество найденных блоков: 20
Запрос прошел успешно!
Количество найденных блоков: 20
Запрос прошел успешно!
Количество найденных блоков: 20
Запрос прошел успешно!
Количество найденных блоков: 6


In [34]:
print(phones_list)

[]


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [18]:
df = pd.DataFrame({'phones': phones_list})

In [0]:
df.loc[:, 'rating'] = df['rating'].str.extract(r'^([0-9\,]+)\s')
df.loc[:, 'rating'] = df['rating'].str.replace(',', '.').astype(float)

In [20]:
dt = datetime.datetime.today().strftime("%y_%m_%d")

In [21]:
df.to_csv('phones_from_google_',dt,'.csv', index=False)

TypeError: "delimiter" must be a 1-character string